In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import src.gen_source as gen_source
from guppy import hpy 
import mydab

import tcp_async
import tcp_sync

from live_analyse_py import live_analyse_py

In [ ]:
try:
    __IPYTHON__
    reload(tcp_async)
    reload(tcp_sync)
    reload(gen_source)
    reload(mydab)
except:
    pass

In [ ]:
sync = tcp_sync.UhdSyncMsg(packet_size=4*8192,
                           packet_type="".join(["f"]*8192))
async = tcp_async.UhdAsyncMsg()

In [ ]:
top = live_analyse_py()

In [ ]:
samp_rate = 4000000 #samples / second
frequency_0  = samp_rate/10. #cycles / second
frequency_1  = samp_rate/9. #cycles / second
fft_size = 8192

In [ ]:
top.start()

In [ ]:
sync.has_msg()

In [ ]:
import peakutils
def score(msg, frequency_0, frequency_1, samp_rate):
    frequency_0_idx = np.round(frequency_0 / samp_rate * fft_size).astype(int)
    frequency_1_idx = np.round(frequency_1 / samp_rate * fft_size).astype(int)
    frequency_idx_diff = frequency_1_idx - frequency_0_idx
    
    start = frequency_0_idx % frequency_idx_diff + frequency_idx_diff / 2.
    start = np.round(start).astype(int)
    end   = msg.shape[0] - frequency_idx_diff - (msg.shape[0] - start) % frequency_idx_diff
    
    msg_cut = msg[start:end]
    
    msg_cut = msg_cut.reshape(-1, frequency_idx_diff)
    
    msg_peaks = np.max(msg_cut, axis=1)
    #plt.plot(msg_peaks, "ro")
    
    inter_mod = \
    (0.5 * (msg_peaks[8]/msg_peaks[7] + msg_peaks[9]/msg_peaks[10]),
     0.5 * (msg_peaks[8]/msg_peaks[6] + msg_peaks[9]/msg_peaks[11]),
     0.5 * (msg_peaks[8]/msg_peaks[5] + msg_peaks[9]/msg_peaks[12]),
     0.5 * (msg_peaks[8]/msg_peaks[4] + msg_peaks[9]/msg_peaks[13]),
     0.5 * (msg_peaks[8]/msg_peaks[3] + msg_peaks[9]/msg_peaks[14]),
     0.5 * (msg_peaks[8]/msg_peaks[2] + msg_peaks[9]/msg_peaks[15]),
     0.5 * (msg_peaks[8]/msg_peaks[1] + msg_peaks[9]/msg_peaks[16]))
    
    return(inter_mod)


In [ ]:
import itertools
x = np.linspace(-0.1, 0.1, num = 5)
l = list(itertools.product(x,x,x,x,x))
mag = [sum(map(lambda x:x*x,el)) for el in l]
idx = list(np.argsort(mag))
l = list(np.array(l)[idx])
len(l)

In [ ]:
top.set_txgain(84)
sync.has_msg()
msgs = np.array(sync.get_msgs(2))
fft = np.fft.fftshift(msgs[0])
#plt.plot(fft)
#plt.plot((4000 - f, 4000 -f),(-180,-40))

mydab.calc_signal_sholder_ratio(fft, fft_size, samp_rate)

In [ ]:
def meas(i):
    import sys
    sys.stdout.write(str(i) + " ")
    top.set_txgain(i)
    sync.has_msg()
    msgs = np.array(sync.get_msgs(10))
    
    res = []
    for msg in msgs:
        fft = np.fft.fftshift(msg)
        res.append(mydab.calc_signal_sholder_ratio(fft, fft_size, samp_rate))
    return np.mean(res)

In [ ]:
res = [meas(i) for i in range(89)]

In [ ]:
plt.plot(res)
plt.title("DAB sholders")
plt.xlabel("tx gain [db]")
plt.ylabel("signal - sholder [db]")

In [ ]:
top.set_txgain(84)
for el in l:
    t2 = time.time()
    path = gen_source.gen_file(frequency_0=frequency_0,
                               frequency_1=frequency_1,
                               samp_rate=samp_rate,
                               coefs = el,
                               path='./np_twotone',
                               count = 10
                               )
    #a_load = np.fromfile(path, dtype=np.complex64)
    #plt.plot(a_load.real[0:100])
    
    sync.has_msg()
    np.array(sync.get_msgs(2))
    msgs = np.array(sync.get_msgs(5))
    scores = [np.max(score(msg, frequency_0, frequency_1, samp_rate)) for msg in msgs]
    inter_mod_avg = np.mean(scores)
    inter_mod_var = np.var(scores)
    msg = msgs[0]
    
    
    param_str = "_".join(["%.3f" % f for f in el])
    print("inter_mod %.3f +- %.3f, %s" % (inter_mod_avg, inter_mod_var, param_str))
    
    plt.plot(msg)
    plt.savefig("/tmp/out/inter_mod_%.3f_+-%.3f_%s.png" % (inter_mod_avg, inter_mod_var, param_str))
    plt.clf()
    
    a = np.fromfile("./np_twotone", dtype=np.complex64)
    plt.plot(a.real)
    plt.savefig("/tmp/out/inter_mod_%.3f_+-%.3f_%s_wave.png" % (inter_mod_avg, inter_mod_var, param_str))
    plt.clf()
    #h = hpy() 
    #print h.heap()
    print(time.time() - t2)

In [ ]:
sync.stop()
async.stop()
top.stop()
top.wait()

In [ ]:
path = gen_source.gen_file(frequency_0=frequency_0,
                           frequency_1=frequency_1,
                           x1 = 0,
                           x2 = 0,
                           x3 = 0,
                           x4 = 0,
                           samp_rate=samp_rate,
                           path='./np_twotone')
a_load = np.fromfile(path, dtype=np.complex64)
plt.plot(a_load.real[0:100])

In [ ]:
gen_source.gen_file_d(800000, 1000000, 0, 0.5, 0, 0, 4000000, count = 5)

In [ ]:
a2 = np.fromfile("./np_twotone", dtype=np.complex64)

In [ ]:
plt.plot(a.real)
plt.plot(a2.real)